In [9]:
## Save and load models
# code to create the model, and the trained weights, or parameters, for the model
# Installs and imports

!pip install -q pyyaml h5py  
# Required to save models in HDF5 format 
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0-dev20200501


In [10]:
# Get an example dataset

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [11]:
# Define a model

# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Save checkpoints during training
# tf.keras.callbacks.ModelCheckpoint : Checkpoint callback usage
 
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.3315 - accuracy: 0.6142
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 1.2063 - accuracy: 0.6540 - val_loss: 0.7237 - val_accuracy: 0.7910
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4410 - accuracy: 0.8841
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4224 - accuracy: 0.8860 - val_loss: 0.5229 - val_accuracy: 0.8330
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.3102 - accuracy: 0.9087
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2953 - accuracy: 0.9180 - val_loss: 0.4461 - val_accuracy: 0.8600
Epoch 4/10
24/32 [=====================>........] - ETA: 0s - loss: 0.1947 - accuracy: 0.9518
Epoch 00004: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 5

In [15]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4189 - accuracy: 0.0860
Untrained model, accuracy:  8.60%


In [16]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc)) 

32/32 - 0s - loss: 0.3864 - accuracy: 0.8750
Restored model, accuracy: 87.50%


In [17]:
# Checkpoint callback options

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [18]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest 

'training_2\\cp-0050.ckpt'

In [19]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
 

32/32 - 0s - loss: 0.4837 - accuracy: 0.8720
Restored model, accuracy: 87.20%


In [20]:
# Manually save weights

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


32/32 - 0s - loss: 0.4837 - accuracy: 0.8720
Restored model, accuracy: 87.20%


In [21]:
# Save the entire model
# SavedModel format
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1806 - accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4240 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2994 - accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2156 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1519 - accuracy: 0.9650
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [22]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model


'ls' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4220 - accuracy: 0.8590
Restored model, accuracy: 85.90%
(1000, 10)


In [26]:
# HDF5 format

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1955 - accuracy: 0.6440
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4360 - accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2840 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2061 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1537 - accuracy: 0.9630


In [28]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary() 

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [29]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4076 - accuracy: 0.8660
Restored model, accuracy: 86.60%
